<br><br><center><h1 style="font-size:3em">Praxisprojekt Datenbanken und SQL</h1></center>
<center><h3>Projekt 3 - ToDo Liste - Anton, Lisa, Maik</b></center>

### Standarteinstellungen

In [1]:
# Importe
import pandas as pd
import psycopg                        
import yaml                            

import sqlalchemy as sa 
import datetime as dt  
from sqlalchemy import text


In [4]:
# Standart-config-Datei laden
with open('..\\config.yaml', 'r') as file:
    config = yaml.load(file, Loader=yaml.BaseLoader)


In [20]:
config

{'host': 'localhost',
 'port': '5432',
 'user': 'postgres',
 'password': 'DataCraft',
 'dbname': 'dvdrental'}

### Neue Datenbank erstellen

In [18]:
# Connection-String
connection_string = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/" 

# Verbindungen aufbauen
sa_eng = sa.create_engine(connection_string, 
                          isolation_level='AUTOCOMMIT')

# Datenbank todo erstellen
with sa_eng.connect() as con:            
    con.execute(text('''CREATE DATABASE todo'''))

In [ ]:
# alte Verbindung trennen
sa_eng.dispose()

### Verbindung zur todo-Datenbank

In [33]:
# Verbindung zur todo-Datenbank erstellen
con_string_todo = f'postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo'

# Verbindungen aufbauen
sa_eng = sa.create_engine(con_string_todo, 
                          isolation_level='AUTOCOMMIT')



### Tabellen erstellen und definieren

In [26]:
# TABELLE - PROJEKTE
# projekt_id serial PRIMARY KEY,
# projekt_name text NOT NULL,
# beschreibung text,
# startdatum date NOT NULL
# enddatum date NOT NULL

with sa_eng.connect() as con:            
    con.execute(text('''
                    CREATE TABLE projekte(
                        projekt_id SERIAL PRIMARY KEY,
                        projekt_name TEXT NOT NULL,
                        beschreibung TEXT,
                        startdatum DATE NOT NULL,
                        enddatum DATE NOT NULL);
                     '''))

# TABELLE - TEAM
# mitarbeiter_id PRIMARY KEY
# vorname
# nachname
# rolle (projektleitung, entwicklung, design,...)
# email

with sa_eng.connect() as con:            
    con.execute(text('''
        CREATE TABLE team(
            mitarbeiter_id SERIAL PRIMARY KEY,
            vorname TEXT NOT NULL,
            nachname TEXT NOT NULL,
            rolle TEXT,
            email TEXT NOT NULL
        );
    '''))

# TABELLE - AUFGABEN
# aufgaben_id int PRIMARY KEY
# aufgaben_name
# projekt_id int FOREIGN KEY
# aufgaben_beschreibung text
# mitarbeiter_id FOREIGN KEY
# status text

with sa_eng.connect() as con:            
    con.execute(text('''
        CREATE TABLE aufgaben(
            aufgaben_id SERIAL PRIMARY KEY,
            aufgaben_name TEXT NOT NULL,
            projekt_id INT,
            aufgaben_beschreibung TEXT,
            mitarbeiter_id INT,
            status TEXT,
            FOREIGN KEY (projekt_id) REFERENCES projekte(projekt_id),
            FOREIGN KEY (mitarbeiter_id) REFERENCES team(mitarbeiter_id)
        );
    '''))

### Klassen und Funktionen

In [22]:
class Projekt:
    def __init__(self, projekt_id, projekt_name, beschreibung, startdatum, enddatum):
        self.projekt_id = projekt_id
        self.projekt_name = projekt_name
        self.beschreibung = beschreibung
        self.startdatum = startdatum
        self.enddatum = enddatum
    
        # Standart-config-Datei laden
        with open('..\\config.yaml', 'r') as file:
            config = yaml.load(file, Loader=yaml.BaseLoader)

        # Connection-String
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl Tabelle
        meta = sa.MetaData(bind=self.sa_eng)
        self.projekte_table = sa.Table('projekte', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren
        ins = self.projekte_table.insert().values(
            projekt_name=self.projekt_name,
            beschreibung=self.beschreibung,
            startdatum=self.startdatum,
            enddatum=self.enddatum
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(ins)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Einfügen in die Datenbank: {e}")

    # Definition UPDATE 
    def update(self):
        # Einträge definieren
        upd = self.projekte_table.update().where(self.projekte_table.c.projekt_id == self.projekt_id).values(
                    projekt_name=self.projekt_name,
                    beschreibung=self.beschreibung,
                    startdatum=self.startdatum,
                    enddatum=self.enddatum
                    )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(upd)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Aktualisieren in der Datenbank: {e}")

    # Definition DELETE 
    def delete(self):
        # Einträge definieren
        delete_projekt = self.projekte_table.delete().where(self.projekte_table.c.projekt_id == self.projekt_id)
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(delete_projekt)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Löschen aus der Datenbank: {e}")

In [ ]:
class Team:
    def __init__(self, mitarbeiter_id, vorname, nachname, rolle, email):
        self.mitarbeiter_id = mitarbeiter_id
        self.vorname = vorname
        self.nachname = nachname
        self.rolle = rolle
        self.email = email
        
        # Standard-config-Datei laden und Verbindung zur Datenbank herstellen
        with open('..\\config.yaml', 'r') as file:
            config = yaml.safe_load(file)

        # Connection-String für SQLAlchemy
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank herstellen
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl der Tabelle
        meta = sa.MetaData(bind=self.sa_eng)
        self.team_table = sa.Table('team', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren und einfügen
        ins = self.team_table.insert().values(
            vorname=self.vorname,
            nachname=self.nachname,
            rolle=self.rolle,
            email=self.email
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(ins)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Einfügen in die Datenbank: {e}")

    # Definition UPDATE 
    def update(self):
        # Einträge definieren und aktualisieren
        upd = self.team_table.update().where(self.team_table.c.mitarbeiter_id == self.mitarbeiter_id).values(
            vorname=self.vorname,
            nachname=self.nachname,
            rolle=self.rolle,
            email=self.email
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(upd)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Aktualisieren in der Datenbank: {e}")

    # Definition DELETE 
    def delete(self):
        # Einträge definieren und löschen
        delete_teammitglied = self.team_table.delete().where(self.team_table.c.mitarbeiter_id == self.mitarbeiter_id)
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(delete_teammitglied)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Löschen aus der Datenbank: {e}")

In [ ]:
class Aufgaben:
    def __init__(self, aufgaben_id, aufgaben_name, projekt_id, aufgaben_beschreibung, mitarbeiter_id, status):
        self.aufgaben_id = aufgaben_id
        self.aufgaben_name = aufgaben_name
        self.projekt_id = projekt_id
        self.aufgaben_beschreibung = aufgaben_beschreibung
        self.mitarbeiter_id = mitarbeiter_id
        self.status = status
        
        # Standard-config-Datei laden und Verbindung zur Datenbank herstellen
        with open('..\\config.yaml', 'r') as file:
            config = yaml.safe_load(file)

        # Connection-String für SQLAlchemy
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank herstellen
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl der Tabelle
        meta = sa.MetaData(bind=self.sa_eng)
        self.aufgaben_table = sa.Table('aufgaben', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren und einfügen
        ins = self.aufgaben_table.insert().values(
            aufgaben_name=self.aufgaben_name,
            projekt_id=self.projekt_id,
            aufgaben_beschreibung=self.aufgaben_beschreibung,
            mitarbeiter_id=self.mitarbeiter_id,
            status=self.status
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(ins)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Einfügen in die Datenbank: {e}")

    # Definition UPDATE 
    def update(self):
        # Einträge definieren und aktualisieren
        upd = self.aufgaben_table.update().where(self.aufgaben_table.c.aufgaben_id == self.aufgaben_id).values(
            aufgaben_name=self.aufgaben_name,
            projekt_id=self.projekt_id,
            aufgaben_beschreibung=self.aufgaben_beschreibung,
            mitarbeiter_id=self.mitarbeiter_id,
            status=self.status
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(upd)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Aktualisieren in der Datenbank: {e}")

    # Definition DELETE 
    def delete(self):
        # Einträge definieren und löschen
        delete_aufgabe = self.aufgaben_table.delete().where(self.aufgaben_table.c.aufgaben_id == self.aufgaben_id)
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(delete_aufgabe)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Löschen aus der Datenbank: {e}")

### Trigger und Log-Tabellen

In [34]:
# LOG Tabellen
with sa_eng.connect() as con:    
    # Log Tabelle Porjekte        
    con.execute(text('''
         CREATE TABLE projekte_log(
            log_projekt_id SERIAL PRIMARY KEY, 
            projekt_id INT NOT NULL,
            typ TEXT NOT NULL,
            projekt_name TEXT,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))

    # Log Tabelle Aufgaben
    con.execute(text('''
         CREATE TABLE aufgaben_log(
            log_aufgaben_id SERIAL PRIMARY KEY, 
            aufgaben_id INT NOT NULL,
            projekt_id INT NOT NULL,
            typ TEXT NOT NULL,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))
    
    # Log Tabelle Team
    con.execute(text('''
         CREATE TABLE team_log(
            log_team_id SERIAL PRIMARY KEY, 
            mitarbeiter_id INT NOT NULL,
            typ TEXT NOT NULL,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))

#### Trigger - Funktionen Projekt

In [35]:
# TRIGGER FUNKTIONEN projekt

with sa_eng.connect() as con:            
    # Trigger-Funktion für INSERT
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name, aenderung)
            VALUES (NEW.projekt_id, 'INSERT', NEW.projekt_name, CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

    # Trigger-Funktion für UPDATE
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name,  aenderung)
            VALUES (OLD.projekt_id, 'UPDATE', OLD.projekt_name, CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

    # Trigger-Funktion für DELETE
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name, aenderung)
            VALUES (OLD.projekt_id, 'DELETE', OLD.projekt_name, CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    ''')) 

In [36]:
# TRIGGER Tabelle Projekte

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_insert_trigger
        AFTER INSERT ON projekte
        FOR EACH ROW
        EXECUTE FUNCTION log_projekte_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_update_trigger
        AFTER UPDATE ON projekte
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_projekte_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_delete_trigger
        AFTER DELETE ON projekte
        FOR EACH ROW
        EXECUTE FUNCTION log_projekte_delete();
    '''))

#### Trigger - Funktionen Aufgaben

In [37]:
# TRIGGER FUNKTIONEN für Aufgaben

# Trigger-Funktion für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (NEW.aufgaben_id, NEW.projekt_id, 'INSERT', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (OLD.aufgaben_id, OLD.projekt_id, 'UPDATE', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (OLD.aufgaben_id, OLD.projekt_id, 'DELETE', CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    '''))


In [38]:
# TRIGGER Tabelle Aufgaben

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_insert_trigger
        AFTER INSERT ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_update_trigger
        AFTER UPDATE ON aufgaben
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_aufgaben_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_delete_trigger
        AFTER DELETE ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_delete();
    '''))

#### Trigger - Funktionen Team

In [39]:
# TRIGGER FUNKTIONEN für Team

# Trigger-Funktion für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (NEW.mitarbeiter_id, 'INSERT', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (OLD.mitarbeiter_id, 'UPDATE', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (OLD.mitarbeiter_id, 'DELETE', CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    '''))

In [40]:
# TRIGGER für Tabelle Team

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_insert_trigger
        AFTER INSERT ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_update_trigger
        AFTER UPDATE ON aufgaben
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_aufgaben_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_delete_trigger
        AFTER DELETE ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_delete();
    '''))

ProgrammingError: (psycopg2.errors.DuplicateObject) FEHLER:  Trigger »log_aufgaben_insert_trigger« für Relation »aufgaben« existiert bereits

[SQL: 
        CREATE TRIGGER log_aufgaben_insert_trigger
        AFTER INSERT ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_insert();
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)

### Beispiel-Daten in Tabellen einfügen

In [15]:
# BEISPIELDATEN Tabelle PROJEKTE

projekt_1 = ('Northwind Datenanalyse', 
            'Verbesserung der Datenverarbeitung und -auswertung für Northwind Traders mithilfe einer SQL-Datenbank.',
            '2024-01-01', '2024-02-15')
                    
projekt_2 = ('Leihräder-Analyse', 
            'Umfassende Datenanalyse der Leihräder-Nutzung in San Francisco.',
            '2024-02-01', '2024-06-30')

projekt_3 = ('ToDo-Liste', 
            'Entwicklung einer ToDo-Listen Anwendung in Python mit SQL-Datenbankverknüpfung.',
            '2024-03-01', '2024-05-31')
             
projekt_4 = ('FIFA Fußball', 
            'Datenanalyse aller professionellen, europäischen Fußballspiele von 2008 bis 2016.',
            '2024-04-01', '2024-09-30')

In [12]:
# BEISPIELDATEN Tabelle TEAM

team_1 = 'Sarah', 'Müller', 'Projektleiterin', 'sarah.mueller@firma.de'

team_2 = 'Johannes', 'Schmidt', 'Datenanalyst', 'johannes.schmidt@firma.de'

team_3 = 'Derya', 'Toptas', 'Datenwissenschaftler', 'derya.toptas@firma.de'

team_4 = 'Julian', 'Schneider', 'Dateningenieur', 'julian.schneider@firma.de'

team_5 = 'Laura', 'Fischer', 'Business-Analystin', 'laura.fischer@firma.de'

In [13]:
# BEISPIELDATEN Tabelle AUFGABEN

aufgabe_1 = ('Northwind DB Aufbau', 
            'Erstellung und Verwaltung der SQL-Datenbank für Northwind.',
            4, 'In Bearbeitung')

aufgabe_2 = ('Northwind Analyse', 
            'Durchführung der Datenanalyse und Beantwortung der Fragen.',
            2, 'In Bearbeitung')

aufgabe_3 = ('Leihräder DB Aufbau', 
                        'Import der Leihräder-Daten in die PostgreSQL-Datenbank und Einrichtung von Schlüsseln.',
                        4, 'In Bearbeitung')
aufgabe_4 = ('Leihräder Analyse', 
                        'Analyse der Leihräder-Daten und Erstellung von Berichten.',
                        2, 'In Bearbeitung')

aufgabe_5 = ('ToDo-Liste Entwicklung', 
                        'Entwicklung der ToDo-Listen Anwendung in Python und Verknüpfung mit SQL-Datenbank.',
                        3, 'In Bearbeitung')
             
aufgabe_6 = ('ToDo-Liste Interface', 
                        'Entwicklung eines Interfaces zur Bedienung der ToDo-Listen Anwendung.',
                        3, 'Offen')
            
aufgabe_7 = ('FIFA DB Aufbau', 
                        'Übertragung des FIFA-Datensatzes in eine neue PostgreSQL-Datenbank.',
                        4, 'Offen')

aufgabe_8 = ('FIFA Analyse', 
                        'Durchführung der Datenanalyse und Beantwortung der Fragen.',
                        2, 'Offen')
             
aufgabe_9 = ('Ergebnispräsentation', 
                        'Erstellung von Präsentationen zur Vorstellung der Analyseergebnisse.',
                        5, 'Offen')